In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
# import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [2]:
import numpy as np
data = np.load("./model_data.npy", allow_pickle = True)

In [3]:
np.random.shuffle(data)
len_training_data = int(len(data)//1.25)
len_training_data
data[0][1]

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [4]:
x_train = []
y_train = np.zeros(shape = (len_training_data, 40000), dtype = np.int8)
for i, item in enumerate(data[:len_training_data]):
    x_train.append(" ".join(item[0]))
    y_train[i] = item[1]

# x_test = data[len_training_data:]
# y_test = data[len_training_data:][1]

In [13]:
model_inputs = keras.Input(shape=(512,))
dense = layers.Dense(1024, activation="elu")
x = dense(model_inputs)
x = layers.Dense(1024, activation = "elu")(x)
x = layers.Dense(1024, activation = "elu")(x)
outputs = layers.Dense(40000, activation = "softmax")(x)

decoder_model = keras.Model(inputs=model_inputs, outputs=outputs)
decoder_model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)
decoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_7 (Dense)              (None, 40000)             41000000  
Total params: 43,624,512
Trainable params: 43,624,512
Non-trainable params: 0
_________________________________________________________________


In [6]:
x_train = embed(x_train)
x_train

<tf.Tensor: shape=(308411, 512), dtype=float32, numpy=
array([[ 2.0769199e-02,  9.0376036e-03,  2.0841695e-02, ...,
        -5.2537758e-02, -5.6964193e-02, -6.4960741e-02],
       [ 2.9190961e-02,  9.1577210e-02, -3.7631325e-02, ...,
        -6.2494171e-03,  8.5921936e-02, -3.7803978e-02],
       [ 5.2836340e-02, -3.8392313e-02, -2.4506299e-02, ...,
         3.7886109e-02, -2.7161047e-02,  3.6496837e-02],
       ...,
       [ 3.6472607e-02,  3.8332153e-02,  9.9520655e-03, ...,
         3.7644118e-02, -2.8622758e-02, -4.7557620e-05],
       [ 2.9890118e-02, -7.6035239e-02, -7.6189741e-02, ...,
         1.2515340e-02, -7.2985671e-02, -6.9292575e-02],
       [ 5.6249116e-02, -6.2787525e-02, -2.7517671e-02, ...,
        -5.0218619e-02, -1.6206281e-02,  2.5190677e-02]], dtype=float32)>

In [11]:
print(x_train.shape)
new = y_train.reshape((308411, 1, 40000))
new.shape

(308411, 512)


(308411, 1, 40000)

In [20]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [14]:
history = decoder_model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0)

Epoch 1/2
  97/4819 [..............................] - ETA: 24:00 - loss: 10.0472 - accuracy: 0.0065

KeyboardInterrupt: 